In [1]:
import re

from bs4 import BeautifulSoup 
import gensim
from gensim.models import LdaModel
from gensim.corpora import Dictionary, MmCorpus
from langdetect import detect
import numpy as np
import pandas as pd
try:
    import spacy
except: 
    !pip install spacy #--user
finally:
    import spacy
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

#import bokeh.plotting as bp
#from bokeh.plotting import save
#from bokeh.models import HoverTool

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

#!pip install pyLDAvis --user 
import pyLDAvis.sklearn
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

%config InlineBackend.figure_format = 'retina'

import warnings
warnings.filterwarnings("ignore")

## First LDA attempt. Note: intresting clusterization of badwods!

In [2]:
meetup_topcats_samp = pd.read_csv('processed_description_sine_error.csv')

In [3]:
meetup_topcats_samp

,Unnamed: 0,description,category_bin,lang_ok,processed_description
0,0,<p>FOOD AND FELLOWSHIP | 5 PM</p> \n<p>CONCERT...,27,english,food fellowship pm concert pm may th join us m...
1,1,<p><b>【WhyNot!?JAPAN + MeetUp Collaboration Ev...,28,english,whynot japan meetup collabor event peopl come ...
2,2,<p>TENTATIVE<br></p>\n<p><br></p>\n<p>10.30 - ...,31,english,tentat introduct orient session break network ...
3,3,"<p><span>Hey explorersssss, what's up?</span><...",15,english,hey explorersssss say night view hong kong vic...
4,4,"<p>Free, unauditioned, collaborative pop choir...",18,english,free unaudit collabor pop choir identifi women...
5,6,"<p>We provide a forum to learn about, promote ...",3,english,provid forum learn promot grow member busi
6,7,<p><b>【WhyNot!?JAPAN + MeetUp Collaboration Ev...,28,english,whynot japan meetup collabor event peopl come ...
7,8,<p>This is an introductory meeting to get to k...,31,english,introductori meet get know fellow coder share ...
8,9,"<p>The Future of the IoT with Rajeev Suri, CEO...",31,english,futur iot rajeev suri ceo kathrin buvac chief ...
9,10,"<p><img src=""https://secure.meetupstatic.com/p...",13,english,breakthrough weight loss wk workshop lose weig...


In [24]:
meetup_topcats_samp=meetup_topcats_samp[meetup_topcats_samp.processed_description.isna()!=True]

### Vectorization

In [25]:
tf_vectorizer = TfidfVectorizer(analyzer = "word", stop_words='english', max_features = 1000)

In [26]:
vectorized = tf_vectorizer.fit_transform(meetup_topcats_samp.processed_description)
vectorized.shape

vectorized.A

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

### LDA model with 33 topics

In [29]:
lda_model = LatentDirichletAllocation(n_components=33, random_state=0, n_jobs=4)

In [30]:
X_topics = lda_model.fit_transform(vectorized)

In [31]:
word_dict = {}
for topic_idx, topic in enumerate(lda_model.components_):
    word_dict[topic_idx] = [tf_vectorizer.get_feature_names()[i] for i in topic.argsort()[:-10 - 1:-1]]

pd.DataFrame(word_dict).T

,0,1,2,3,4,5,6,7,8,9
0,workout,fit,class,train,bodi,weight,session,exercis,pm,rd
1,game,play,bring,board,night,fun,drink,pm,prize,tabl
2,event,danc,ticket,band,pm,parti,music,night,drink,test
3,heal,energi,reiki,bodi,medit,class,practic,spiritu,sound,yoga
4,menu,food,cook,restaur,dish,eat,dinner,meal,pleas,soon
5,ride,hike,trail,mile,bike,park,rout,road,pace,rider
6,hike,particip,respons,event,agre,organ,risk,injuri,liabil,safeti
7,busi,network,market,event,meet,profession,entrepreneur,group,help,grow
8,paddl,boat,water,kayak,beach,river,climb,fish,bring,park
9,music,perform,comedi,song,sing,festiv,ticket,pm,artist,art


## LDA VIZ

In [39]:
pyLDAvis.sklearn.prepare(lda_model, vectorized, tf_vectorizer, mds='pcoa')

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
28     0.128261  0.058941       1        1  6.366143
31     0.102519 -0.033107       2        1  4.865828
5     -0.105980 -0.028726       3        1  4.746153
7      0.104800  0.056402       4        1  4.693248
26    -0.102113 -0.021894       5        1  4.411936
3      0.098515 -0.081936       6        1  4.404421
20    -0.038246 -0.049647       7        1  3.860723
2     -0.058266  0.011721       8        1  3.812595
10    -0.048295  0.004562       9        1  3.655395
29     0.036652  0.013909      10        1  3.452406
9     -0.007269 -0.000617      11        1  3.296292
24     0.082079 -0.070896      12        1  3.254441
19     0.046445 -0.077445      13        1  3.149847
23     0.047100  0.028718      14        1  2.924899
15     0.020639 -0.119990      15        1  2.866697
27     0.059862  0.024618      16        1  2.805954
13    -0.031284  0.000633      17        1  2.774702
1     -0.048505 -0.019869      18        1  2.773721
6     -0.074506 -0.032682      19        1  2.594311
8     -0.103496 -0.017311      20        1  2.585426
25    -0.043960  0.007231      21        1  2.530465
22     0.031714 -0.023478      22        1  2.502582
18     0.019795 -0.032947      23        1  2.431989
4     -0.042546  0.000309      24        1  2.370288
14     0.007764  0.016368      25        1  2.244778
17     0.101158  0.044362      26        1  2.178596
11    -0.057245  0.000763      27        1  2.169356
0     -0.001725 -0.085812      28        1  2.135865
21    -0.055188 -0.029418      29        1  2.094019
12     0.053720 -0.060519      30        1  1.788669
32     0.195599  0.283804      31        1  1.463531
30    -0.269319  0.228361      32        1  1.443493
16    -0.048681  0.005594      33        1  1.351229, topic_info=    Category         Freq        Term        Total  loglift  logprob
775  Default  2034.000000        sail  2034.000000  30.0000  30.0000
156  Default  4558.000000       class  4558.000000  29.0000  29.0000
370  Default  3013.000000        game  3013.000000  28.0000  28.0000
997  Default  2063.000000        yoga  2063.000000  27.0000  27.0000
557  Default  2172.000000       medit  2172.000000  26.0000  26.0000
777  Default  1053.000000      sailor  1053.000000  25.0000  25.0000
593  Default  1601.000000      nation  1601.000000  24.0000  24.0000
416  Default  2179.000000        hike  2179.000000  23.0000  23.0000
228  Default  2545.000000        danc  2545.000000  22.0000  22.0000
758  Default  2027.000000        ride  2027.000000  21.0000  21.0000
666  Default  2782.000000        play  2782.000000  20.0000  20.0000
667  Default  1458.000000      player  1458.000000  19.0000  19.0000
116  Default  2595.000000        busi  2595.000000  18.0000  18.0000
853  Default   854.000000     startup   854.000000  17.0000  17.0000
772  Default  1861.000000         run  1861.000000  16.0000  16.0000
982  Default  1220.000000        wine  1220.000000  15.0000  15.0000
921  Default  1493.000000       trail  1493.000000  14.0000  14.0000
588  Default   904.000000        movi   904.000000  13.0000  13.0000
988  Default   857.000000     workout   857.000000  12.0000  12.0000
600  Default  1970.000000     network  1970.000000  11.0000  11.0000
634  Default   824.000000       paint   824.000000  10.0000  10.0000
404  Default  1228.000000        heal  1228.000000   9.0000   9.0000
908  Default  2721.000000      ticket  2721.000000   8.0000   8.0000
559  Default  5242.000000        meet  5242.000000   7.0000   7.0000
568  Default  1436.000000        mile  1436.000000   6.0000   6.0000
99   Default  1569.000000        bodi  1569.000000   5.0000   5.0000
213  Default   922.000000       court   922.000000   4.0000   4.0000
994  Default   505.000000       yacht   505.000000   3.0000   3.0000
346  Default  1217.000000         fit  1217.000000   2.0000   2.0000
187  Default   497.000000   comp

In [40]:
pyLDAvis.sklearn.prepare(lda_model, vectorized, tf_vectorizer, mds='mmds')

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
28    -0.137501  0.118587       1        1  6.366143
31    -0.034898  0.135153       2        1  4.865828
5      0.176035 -0.048475       3        1  4.746153
7     -0.130118  0.061579       4        1  4.693248
26     0.135580 -0.070762       5        1  4.411936
3      0.017815  0.192400       6        1  4.404421
20     0.120885  0.027564       7        1  3.860723
2     -0.074113 -0.133237       8        1  3.812595
10     0.059730 -0.013626       9        1  3.655395
29    -0.070026  0.013159      10        1  3.452406
9     -0.043672 -0.027663      11        1  3.296292
24     0.006240  0.161089      12        1  3.254441
19     0.055835  0.115558      13        1  3.149847
23    -0.082576  0.072222      14        1  2.924899
15     0.110463  0.157131      15        1  2.866697
27    -0.054902  0.086463      16        1  2.805954
13    -0.011959 -0.047193      17        1  2.774702
1     -0.037473 -0.123086      18        1  2.773721
6      0.166686 -0.091499      19        1  2.594311
8      0.113906 -0.049501      20        1  2.585426
25    -0.030349 -0.095863      21        1  2.530465
22    -0.120306 -0.027398      22        1  2.502582
18     0.003274  0.031957      23        1  2.431989
4      0.014794 -0.123639      24        1  2.370288
14    -0.152151 -0.065021      25        1  2.244778
17    -0.164939  0.049032      26        1  2.178596
11     0.011297 -0.090583      27        1  2.169356
0      0.125731  0.098110      28        1  2.135865
21     0.062655 -0.127716      29        1  2.094019
12    -0.015868 -0.223418      30        1  1.788669
32    -0.293974  0.250257      31        1  1.463531
30     0.432064 -0.036130      32        1  1.443493
16    -0.158166 -0.175450      33        1  1.351229, topic_info=    Category         Freq        Term        Total  loglift  logprob
775  Default  2034.000000        sail  2034.000000  30.0000  30.0000
156  Default  4558.000000       class  4558.000000  29.0000  29.0000
370  Default  3013.000000        game  3013.000000  28.0000  28.0000
997  Default  2063.000000        yoga  2063.000000  27.0000  27.0000
557  Default  2172.000000       medit  2172.000000  26.0000  26.0000
777  Default  1053.000000      sailor  1053.000000  25.0000  25.0000
593  Default  1601.000000      nation  1601.000000  24.0000  24.0000
416  Default  2179.000000        hike  2179.000000  23.0000  23.0000
228  Default  2545.000000        danc  2545.000000  22.0000  22.0000
758  Default  2027.000000        ride  2027.000000  21.0000  21.0000
666  Default  2782.000000        play  2782.000000  20.0000  20.0000
667  Default  1458.000000      player  1458.000000  19.0000  19.0000
116  Default  2595.000000        busi  2595.000000  18.0000  18.0000
853  Default   854.000000     startup   854.000000  17.0000  17.0000
772  Default  1861.000000         run  1861.000000  16.0000  16.0000
982  Default  1220.000000        wine  1220.000000  15.0000  15.0000
921  Default  1493.000000       trail  1493.000000  14.0000  14.0000
588  Default   904.000000        movi   904.000000  13.0000  13.0000
988  Default   857.000000     workout   857.000000  12.0000  12.0000
600  Default  1970.000000     network  1970.000000  11.0000  11.0000
634  Default   824.000000       paint   824.000000  10.0000  10.0000
404  Default  1228.000000        heal  1228.000000   9.0000   9.0000
908  Default  2721.000000      ticket  2721.000000   8.0000   8.0000
559  Default  5242.000000        meet  5242.000000   7.0000   7.0000
568  Default  1436.000000        mile  1436.000000   6.0000   6.0000
99   Default  1569.000000        bodi  1569.000000   5.0000   5.0000
213  Default   922.000000       court   922.000000   4.0000   4.0000
994  Default   505.000000       yacht   505.000000   3.0000   3.0000
346  Default  1217.000000         fit  1217.000000   2.0000   2.0000
187  Default   497.000000   comp

In [41]:
pyLDAvis.sklearn.prepare(lda_model, vectorized, tf_vectorizer, mds='tsne') #remember: topic 25 is cool!

PreparedData(topic_coordinates=                x           y  topics  cluster      Freq
topic                                                   
28      44.396221 -158.286667       1        1  6.366143
31      74.199814 -105.274780       2        1  4.865828
5      -64.996307 -125.206955       3        1  4.746153
7       -8.929233 -142.446625       4        1  4.693248
26     -72.903015  -73.529213       5        1  4.411936
3      134.855469  -97.338646       6        1  4.404421
20       9.727978  140.673340       7        1  3.860723
2      -82.746437   27.309099       8        1  3.812595
10     -29.524336   -7.530567       9        1  3.655395
29      19.099943   24.984331      10        1  3.452406
9      -76.740730   74.921051      11        1  3.296292
24     158.805710  -45.863331      12        1  3.254441
19     132.541687   10.270113      13        1  3.149847
23      13.546177  -96.956848      14        1  2.924899
15    -130.351501 -108.197014      15        1  2.866697
27    -128.989594  -40.480648      16        1  2.805954
13      18.859062   74.633270      17        1  2.774702
1      -27.357580   97.212761      18        1  2.773721
6       95.181503  -47.279751      19        1  2.594311
8       15.682925  -21.671074      20        1  2.585426
25     -75.684120  -20.644985      21        1  2.530465
22     -22.418400  -60.499989      22        1  2.502582
18      44.149590  -57.145805      23        1  2.431989
4       67.573616   51.673336      24        1  2.370288
14      67.814575  112.970230      25        1  2.244778
17     127.618164   72.587013      26        1  2.178596
11     -29.734344   43.342712      27        1  2.169356
0      -68.053581  143.521194      28        1  2.135865
21    -137.214417   24.783062      29        1  2.094019
12     115.728813 -155.701706      30        1  1.788669
32     -11.779943 -211.990601      31        1  1.463531
30      73.336029    0.436334      32        1  1.443493
16    -133.175049   96.597855      33        1  1.351229, topic_info=    Category         Freq        Term        Total  loglift  logprob
775  Default  2034.000000        sail  2034.000000  30.0000  30.0000
156  Default  4558.000000       class  4558.000000  29.0000  29.0000
370  Default  3013.000000        game  3013.000000  28.0000  28.0000
997  Default  2063.000000        yoga  2063.000000  27.0000  27.0000
557  Default  2172.000000       medit  2172.000000  26.0000  26.0000
777  Default  1053.000000      sailor  1053.000000  25.0000  25.0000
593  Default  1601.000000      nation  1601.000000  24.0000  24.0000
416  Default  2179.000000        hike  2179.000000  23.0000  23.0000
228  Default  2545.000000        danc  2545.000000  22.0000  22.0000
758  Default  2027.000000        ride  2027.000000  21.0000  21.0000
666  Default  2782.000000        play  2782.000000  20.0000  20.0000
667  Default  1458.000000      player  1458.000000  19.0000  19.0000
116  Default  2595.000000        busi  2595.000000  18.0000  18.0000
853  Default   854.000000     startup   854.000000  17.0000  17.0000
772  Default  1861.000000         run  1861.000000  16.0000  16.0000
982  Default  1220.000000        wine  1220.000000  15.0000  15.0000
921  Default  1493.000000       trail  1493.000000  14.0000  14.0000
588  Default   904.000000        movi   904.000000  13.0000  13.0000
988  Default   857.000000     workout   857.000000  12.0000  12.0000
600  Default  1970.000000     network  1970.000000  11.0000  11.0000
634  Default   824.000000       paint   824.000000  10.0000  10.0000
404  Default  1228.000000        heal  1228.000000   9.0000   9.0000
908  Default  2721.000000      ticket  2721.000000   8.0000   8.0000
559  Default  5242.000000        meet  5242.000000   7.0000   7.0000
568  Default  1436.000000        mile  1436.000000   6.0000   6.0000
99   Default  1569.000000        bodi  1569.000000   5.0000   5.0000
213  Default   922.000000       court   922.000000   4.0000   4.0000
994  Default   505.000000     